In [1]:
# classification mnist example 
import  os
os.environ['nvcc_path']='/usr/local/cuda-11.0/bin/nvcc'
import jittor as jt  # 将 jittor 引入
from jittor import nn, Module  # 引入相关的模块
import numpy as np
import math 
from jittor import init
if jt.has_cuda:
    jt.flags.use_cuda = 1 # jt.flags.use_cuda 表示是否使用 gpu 训练。
# 如果 jt.flags.use_cuda=1，表示使用GPU训练 如果 jt.flags.use_cuda = 0 表示使用 CPU
from jittor.dataset.mnist import MNIST 
#由于 MNIST 是一个常见的数据集，其数据载入已经被封装进 jittor 所以可以直接调用。
# import matplotlib.pyplot as plt

[i 0404 22:59:29.797014 04 log.cc:351] Load log_sync: 1
[i 0404 22:59:29.827454 04 compiler.py:951] Jittor(1.3.1.56) src: /home/hx-gpu3/anaconda3/envs/mmlab/lib/python3.9/site-packages/jittor
[i 0404 22:59:29.829971 04 compiler.py:952] g++ at /usr/bin/g++(7.5.0)
[i 0404 22:59:29.830405 04 compiler.py:953] cache_path: /home/hx-gpu3/.cache/jittor/jt1.3.1/g++7.5.0/py3.9.10/Linux-5.4.0-99x95/IntelRCoreTMi9x92/default
[i 0404 22:59:29.832758 04 compiler.py:896] Found nvcc(11.0.194) at /usr/local/cuda-11.0/bin/nvcc
[i 0404 22:59:29.868012 04 __init__.py:411] Found gdb(8.1.0) at /usr/bin/gdb.
[i 0404 22:59:29.871191 04 __init__.py:411] Found addr2line(2.30) at /usr/bin/addr2line.
[i 0404 22:59:29.929899 04 compiler.py:1006] cuda key:cu11.0.194_sm_86
[i 0404 22:59:30.026783 04 __init__.py:227] Total mem: 62.74GB, using 16 procs for compiling.
[i 0404 22:59:30.096048 04 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0404 22:59:30.096599 04 jit_compiler.cc:31] Load nvcc_path: /usr/local/cuda-

In [2]:
import gzip
from PIL import Image
from jittor.dataset import Dataset
from jittor_utils.misc import download_url_to_local

class MNIST(Dataset):
    def __init__(self, data_root="./mnist_data/", train=True ,download=True, batch_size=1, shuffle=False):
        # if you want to test resnet etc you should set input_channel = 3, because the net set 3 as the input dimensions
        super().__init__()
        self.data_root = data_root
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.is_train = train
        if download == True:
            self.download_url()

        filesname = [
                "train-images-idx3-ubyte.gz",
                "t10k-images-idx3-ubyte.gz",
                "train-labels-idx1-ubyte.gz",
                "t10k-labels-idx1-ubyte.gz"
        ]
        self.mnist = {}
        if self.is_train:
            with gzip.open(data_root + filesname[0], 'rb') as f:
                self.mnist["images"] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28, 28)
            with gzip.open(data_root + filesname[2], 'rb') as f:
                self.mnist["labels"] = np.frombuffer(f.read(), np.uint8, offset=8)
        else:
            with gzip.open(data_root + filesname[1], 'rb') as f:
                self.mnist["images"] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28, 28)
            with gzip.open(data_root + filesname[3], 'rb') as f:
                self.mnist["labels"] = np.frombuffer(f.read(), np.uint8, offset=8)
        assert(self.mnist["images"].shape[0] == self.mnist["labels"].shape[0])
        self.total_len = self.mnist["images"].shape[0]
        # this function must be called
        self.set_attrs(batch_size = self.batch_size, total_len=self.total_len, shuffle= self.shuffle)
    def __getitem__(self, index):
        img = Image.fromarray (self.mnist['images'][index]) 
        img = np.array (img)
        img = img[np.newaxis, :]
        return np.array((img / 255.0), dtype = np.float32), self.mnist['labels'][index]

    def download_url(self):
        resources = [
            ("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "f68b3c2dcbeaaa9fbdd348bbdeb94873"),
            ("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz", "d53e105ee54ea40749a09fcbcd1e9432"),
            ("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "9fb629c4189551a2d022fa330f9573f3"),
            ("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz", "ec29112dd5afa0611ce80d1b7f02629c")
        ]

        for url, md5 in resources:
            filename = url.rpartition('/')[2]
            download_url_to_local(url, filename, self.data_root, md5)


In [3]:
class Model (Module):
    def __init__ (self):
        super (Model, self).__init__()

        self.relu = nn.Relu()
        self.fc1 = nn.Linear (28*28, 512)
        self.fc2 = nn.Linear (512, 10)
    def execute (self, x) : 
        # it's simliar to forward function in Pytorch 
        x=jt.reshape(x,[x.shape[0],-1])
        
        x = self.fc1 (x)
        x = self.relu(x)
        x = self.fc2 (x)
#         x= nn.softmax(x)
        return x


In [4]:
def train(model, train_loader, optimizer, epoch, losses, losses_idx):
    model.train()
    lens = len(train_loader)
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        outputs = model(inputs)
        loss = nn.cross_entropy_loss(outputs, targets)
        optimizer.step (loss)
        losses.append(loss.numpy()[0])
#         losses_idx.append(epoch * lens + batch_idx)
        
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader) ,
                100. * batch_idx / len(train_loader), loss.numpy()[0]))

def val(model, val_loader, epoch):
    model.eval()
    
    test_loss = 0
    correct = 0
    total_acc = 0
    total_num = 0
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        batch_size = inputs.shape[0]
        outputs = model(inputs)
        pred = np.argmax(outputs.numpy(), axis=1)
        acc = np.sum(targets.numpy()==pred)
        total_acc += acc
        total_num += batch_size
        acc = acc / batch_size
        
        if batch_idx % 1000 == 0:
            print(f'Test Epoch: {epoch} [{batch_idx}/{len(val_loader)}]\tAcc: {acc:.6f}')    	
            print('Test Acc =', total_acc / total_num)
    
batch_size = 32
learning_rate = 1e-3
epochs = 20
losses = []
losses_idx = []
train_loader = MNIST(train=True, batch_size=batch_size, shuffle=True)

val_loader = MNIST(train=False, batch_size=1, shuffle=False)

model = Model ()
optimizer = nn.Adam(model.parameters(), learning_rate)
for epoch in range(epochs):
    train(model, train_loader, optimizer, epoch, losses, losses_idx)


Train Epoch: 0 [0/1875 (0%)]	Loss: 2.308344
Train Epoch: 0 [500/1875 (27%)]	Loss: 0.183193
Train Epoch: 0 [1000/1875 (53%)]	Loss: 0.183329
Train Epoch: 0 [1500/1875 (80%)]	Loss: 0.094363
Train Epoch: 1 [0/1875 (0%)]	Loss: 0.122191
Train Epoch: 1 [500/1875 (27%)]	Loss: 0.073456
Train Epoch: 1 [1000/1875 (53%)]	Loss: 0.051541
Train Epoch: 1 [1500/1875 (80%)]	Loss: 0.207473
Train Epoch: 2 [0/1875 (0%)]	Loss: 0.099142
Train Epoch: 2 [500/1875 (27%)]	Loss: 0.014521
Train Epoch: 2 [1000/1875 (53%)]	Loss: 0.017011
Train Epoch: 2 [1500/1875 (80%)]	Loss: 0.090822
Train Epoch: 3 [0/1875 (0%)]	Loss: 0.039820
Train Epoch: 3 [500/1875 (27%)]	Loss: 0.121215
Train Epoch: 3 [1000/1875 (53%)]	Loss: 0.016069
Train Epoch: 3 [1500/1875 (80%)]	Loss: 0.154921
Train Epoch: 4 [0/1875 (0%)]	Loss: 0.016104
Train Epoch: 4 [500/1875 (27%)]	Loss: 0.087170
Train Epoch: 4 [1000/1875 (53%)]	Loss: 0.002511
Train Epoch: 4 [1500/1875 (80%)]	Loss: 0.007997
Train Epoch: 5 [0/1875 (0%)]	Loss: 0.011221
Train Epoch: 5 [500/1

In [5]:
val(model, val_loader, epoch)

Test Epoch: 19 [0/10000]	Acc: 1.000000
Test Acc = 1.0
Test Epoch: 19 [1000/10000]	Acc: 1.000000
Test Acc = 0.978021978021978
Test Epoch: 19 [2000/10000]	Acc: 1.000000
Test Acc = 0.9690154922538731
Test Epoch: 19 [3000/10000]	Acc: 1.000000
Test Acc = 0.9700099966677774
Test Epoch: 19 [4000/10000]	Acc: 1.000000
Test Acc = 0.971007248187953
Test Epoch: 19 [5000/10000]	Acc: 1.000000
Test Acc = 0.9722055588882224
Test Epoch: 19 [6000/10000]	Acc: 1.000000
Test Acc = 0.9741709715047492
Test Epoch: 19 [7000/10000]	Acc: 1.000000
Test Acc = 0.9762891015569204
Test Epoch: 19 [8000/10000]	Acc: 1.000000
Test Acc = 0.9785026871641045
Test Epoch: 19 [9000/10000]	Acc: 1.000000
Test Acc = 0.9802244195089435


> B32 82s 0.77G  
> B32 78s 0.77G  
> B32 79s 0.77G  
  
> 69s  
> 67s